In [1]:
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import preprocessing as pp
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from importlib import reload

reload(pp)

pd.options.mode.chained_assignment = None

In [40]:
df = pp.load_datasets()
df = pp.prepare_target(df)
df = df.dropna()
df_ribosome = df[df.classification.isin(['ribosome', 'transferase'])]

#df_ribosome[['classification','residueCount', 'resolution', 'structureMolecularWeight','crystallizationTempK', 'densityMatthews', 'densityPercentSol']].sample().head()
df_ribosome.sample()

Complete DataFrame has 471149 lines and 16 columns
Final DataFrame has 380880 lines and 16 columns after removing all classes with less than 5000 items


,structureId,classification,experimentalTechnique,macromoleculeType,residueCount,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,densityMatthews,densityPercentSol,pdbxDetails,phValue,publicationYear,chainId,sequence
435784,5NDW,ribosome,X-RAY DIFFRACTION,Protein#RNA,34100,3.7,6188107.0,"VAPOR DIFFUSION, HANGING DROP",277.0,3.18,61.32,"PEG 20K, KSCN, Mg Acetate, Tris-Acetate, Glyce...",7.0,2017.0,n6,AKQSLDVSSDRRKARKAYFTAPSSQRRVLLSAPLSKELRAQYGIKA...


In [42]:
to_drop = ['structureId', 'chainId', 'sequence', 'pdbxDetails', 'publicationYear', 'crystallizationMethod','experimentalTechnique']
X = df_ribosome
X = X.drop([x for x in to_drop if x in X.columns], axis=1)
prep = pp.PreprocessingTransformer()
X = prep.handle_missing(X)
X = prep.reduce_modalities(X)
X = prep.handle_skewness(X)
X = prep.scale_encode_data(X)

In [60]:
columns_clf = [ 'residueCount', 'resolution', 'structureMolecularWeight',
                'crystallizationTempK', 'densityMatthews', 'densityPercentSol',
                'macromoleculeType_DNA', 'macromoleculeType_Protein',
                'macromoleculeType_RNA', 'phValue_acide', 'phValue_basique',
                'phValue_neutre']

def ml_predict_with_dataframe(df):
    #Open the model file
    model = joblib.load(model_path)
    #make the prediction
    data = df
    #do this before providing the Dataframe
    #data = data.dropna()
    y = 0
    if('classification' in df.columns):
        y = df.classification
        data = df.drop('classification', axis=1)

    to_drop = ['structureId', 'chainId', 'sequence', 'pdbxDetails', 'publicationYear', 'crystallizationMethod','experimentalTechnique']
    X = df
    data = data.drop([x for x in to_drop if x in data.columns], axis=1)
    prep = pp.PreprocessingTransformer()
    #data = prep.handle_missing(data)
    data = prep.reduce_modalities(data)
    data = prep.handle_skewness(data)
    data = prep.scale_encode_data(data)

    for col in columns_clf:
        if col not in data.columns:
            data[col] = 0

    data = data[columns_clf]
    

    return data

In [65]:
df = pd.read_csv('../data/test.csv')
data = df
#do this before providing the Dataframe
data = data.dropna()
y = 0
if('classification' in df.columns):
    y = df.classification
    data = df.drop('classification', axis=1)

to_drop = ['structureId', 'chainId', 'sequence', 'pdbxDetails', 'publicationYear', 'crystallizationMethod','experimentalTechnique']
X = df
data = data.drop([x for x in to_drop if x in data.columns], axis=1)
prep = pp.PreprocessingTransformer()
data = prep.handle_missing(data)
data = prep.reduce_modalities(data)
data = prep.handle_skewness(data)
data = prep.scale_encode_data(data)

for col in columns_clf:
    if col not in data.columns:
        data[col] = 0

data = data[columns_clf]
data.isna().sum()

residueCount                 0
resolution                   0
structureMolecularWeight     0
crystallizationTempK         0
densityMatthews              0
densityPercentSol            0
macromoleculeType_DNA        0
macromoleculeType_Protein    0
macromoleculeType_RNA        0
phValue_acide                0
phValue_basique              0
phValue_neutre               0
dtype: int64

In [ ]:
model = joblib.load(model_path)
df_final = model.predict(df)

In [58]:
df = pd.read_csv('../data/test.csv')

In [61]:
model_path = "../models/prot_clf.joblib"
df = df.dropna()
df = ml_predict_with_dataframe(df)

model opened


In [63]:
df.isna().sum()

residueCount                 12
resolution                   12
structureMolecularWeight     12
crystallizationTempK         12
densityMatthews              12
densityPercentSol            12
macromoleculeType_DNA         0
macromoleculeType_Protein     0
macromoleculeType_RNA         0
phValue_acide                 0
phValue_basique               0
phValue_neutre                0
dtype: int64

In [ ]:
print("Do the prediction: ")


model opened
Do the prediction: 


C:\Users\engizulu\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- macromoleculeType_RNA

  warnings.warn(message, FutureWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').